# Quick Start: Quantum Measurement Coordination

This notebook reproduces the Quick Start examples from the README.

**Goal:** Generate a single quantum measurement trajectory and detect the coordination event.

---

## Setup

In [ ]:
# Standard imports
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# Set plotting defaults
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 11
%matplotlib inline

print("✓ Environment ready")

---

## Step 1: Generate a Measurement Trajectory

We'll simulate a continuous homodyne measurement of a qubit (d=2) using the stochastic master equation.

**Note:** This uses synthetic data. For actual simulations, use:
```python
from simulate_trajectory import generate_measurement_trajectory
```

In [ ]:
def generate_example_trajectory(d=2, T=20, dt=0.01):
    """
    Generate example coordination measures.
    
    In production, replace with actual QuTiP simulation:
        from simulate_trajectory import generate_measurement_trajectory
        from event_detection import compute_coordination_measures
    """
    time = np.arange(0, T, dt)
    
    # Duration: sigmoid rise
    t_peak = 8.0 / np.sqrt(d)
    width = 2.0 * d**(-0.558)
    Phi_d = 1 / (1 + np.exp(-(time - t_peak) / width))
    
    # Frequency: complementary
    Phi_f = 1 - Phi_d
    
    # Add realistic noise
    np.random.seed(42)
    Phi_d += np.random.normal(0, 0.01, len(time))
    Phi_f += np.random.normal(0, 0.01, len(time))
    Phi_d = np.clip(Phi_d, 0, 1)
    Phi_f = np.clip(Phi_f, 0, 1)
    
    # Coordination capacity (agency spike)
    numerator = 4 * np.sqrt(Phi_d * (1 - Phi_d) * Phi_f * (1 - Phi_f))
    denominator = Phi_d + Phi_f + 1e-10
    Phi_a = numerator / denominator
    
    return time, Phi_d, Phi_f, Phi_a

# Generate trajectory for d=2 qubit
time, Phi_d, Phi_f, Phi_a = generate_example_trajectory(d=2, T=20, dt=0.01)

print(f"✓ Generated trajectory with {len(time)} time points")
print(f"  Time span: {time[0]:.1f} to {time[-1]:.1f} μs")
print(f"  Time step: {time[1] - time[0]:.3f} μs")

---

## Step 2: Compute Coordination Measures

The three coordination measures are:
- **Φ_d** (Duration): Actualized correlation between system and apparatus
- **Φ_f** (Frequency): Remaining superposition / future potential
- **Φ_a** (Agency): Coordination capacity marking measurement events

In [ ]:
# Already computed in generate_example_trajectory
# In production, use:
# Phi_d, Phi_f, Phi_a = compute_coordination_measures(rho_traj)

print("Coordination measures at t=10 μs:")
idx = np.argmin(np.abs(time - 10.0))
print(f"  Phi_d (duration)  = {Phi_d[idx]:.3f}")
print(f"  Phi_f (frequency) = {Phi_f[idx]:.3f}")
print(f"  Phi_a (agency)    = {Phi_a[idx]:.3f}")

---

## Step 3: Detect Measurement Event

The measurement event occurs at the **peak of Φ_a** (coordination capacity).

In [ ]:
# Find peak of coordination capacity
peak_idx = np.argmax(Phi_a)
peak_time = time[peak_idx]
peak_value = Phi_a[peak_idx]

print(f"✓ Measurement event detected!")
print(f"  Time: t = {peak_time:.2f} μs")
print(f"  Peak coordination: Phi_a = {peak_value:.3f}")

---

## Step 4: Compute Coordination Action

The **coordination action** S_coord quantifies the total "cost" of establishing system-apparatus correlation:

$$S_{\text{coord}} = \int \Phi_a(t) \, dt$$

In [ ]:
# Integrate coordination capacity
dt = time[1] - time[0]
S_coord = np.trapz(Phi_a, dx=dt)

print(f"\n✓ Coordination action computed!")
print(f"  S_coord = {S_coord:.3f}")
print(f"\nFor comparison:")
print(f"  Expected for d=2: ~{8.98 * 2**(-1.787):.3f} (from scaling law)")
print(f"  π/8 threshold: ~{np.pi/8:.3f}")

---

## Step 5: Visualize the Trajectory

In [ ]:
# Create figure
fig, axes = plt.subplots(2, 1, figsize=(12, 8))

# Top panel: Duration and Frequency
ax1 = axes[0]
ax1.plot(time, Phi_d, '-', color='#2E86AB', linewidth=2.5, 
         label=r'Duration $\Phi_d$ (actualized correlation)')
ax1.plot(time, Phi_f, '-', color='#A23B72', linewidth=2.5,
         label=r'Frequency $\Phi_f$ (remaining superposition)')
ax1.axhline(0.5, color='gray', linestyle='--', linewidth=1, alpha=0.5)
ax1.set_ylabel('Coordination Measures', fontsize=12, fontweight='bold')
ax1.set_xlim(0, time[-1])
ax1.set_ylim(-0.05, 1.05)
ax1.legend(loc='center right', fontsize=10)
ax1.grid(True, alpha=0.3)
ax1.set_title('Quantum Measurement Trajectory (d=2 Qubit)', 
              fontsize=13, fontweight='bold')

# Bottom panel: Coordination Capacity
ax2 = axes[1]
ax2.fill_between(time, 0, Phi_a, alpha=0.3, color='#F77F00')
ax2.plot(time, Phi_a, '-', color='#F77F00', linewidth=2.5,
         label=r'Coordination capacity $\Phi_a$ (agency spike)')
ax2.plot(peak_time, peak_value, 'o', color='#E63946', 
         markersize=12, zorder=5, label='Measurement event')
ax2.axvline(peak_time, color='#E63946', linestyle='--', 
            linewidth=1.5, alpha=0.7)
ax2.set_xlabel('Time (μs)', fontsize=12, fontweight='bold')
ax2.set_ylabel('Coordination Capacity', fontsize=12, fontweight='bold')
ax2.set_xlim(0, time[-1])
ax2.legend(loc='upper right', fontsize=10)
ax2.grid(True, alpha=0.3)

# Add text annotation
ax2.text(0.02, 0.95, 
         f'Coordination action: $S_{{coord}}$ = {S_coord:.3f}\n' + 
         f'Event at t = {peak_time:.2f} μs',
         transform=ax2.transAxes,
         fontsize=11,
         verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.6))

plt.tight_layout()
plt.show()

print("\n✓ Visualization complete!")

---

## Key Insights

### What This Shows:

1. **Duration (Φ_d) rises** as system-apparatus correlation builds up
2. **Frequency (Φ_f) falls** as quantum superposition decays
3. **Coordination capacity (Φ_a) peaks** when these processes are optimally balanced
4. The **peak marks the measurement event** - the discrete moment when the outcome is determined

### Critical Insight:

> **"Don't track state evolution toward completion; detect coordination events through agency spikes."**

Traditional approaches track ρ(t) gradually evolving toward an eigenstate. This framework detects the **discrete coordination event** via the Φ_a spike.

### The Duration Trap:

❌ **Wrong:** Compute Φ_d from system purity Tr[ρ²]  
→ Pure state throughout, purity = 1, no event detected

✓ **Correct:** Compute Φ_d from system-apparatus Fisher information  
→ Captures correlation building, spike visible, event detected

---

## Next Steps

1. **Notebook 02:** Reproduce the dimensional scaling result (S ∝ d^-1.787)
2. **Notebook 03:** Generate all paper figures
3. **Actual simulations:** Replace synthetic data with QuTiP trajectories

---

## Summary Statistics

In [ ]:
print("="*50)
print("QUICK START SUMMARY")
print("="*50)
print(f"\nSystem: d=2 qubit")
print(f"Measurement: Continuous homodyne (γ=1.0 MHz)")
print(f"\nResults:")
print(f"  Event time: t = {peak_time:.2f} μs")
print(f"  Peak coordination: Φ_a = {peak_value:.3f}")
print(f"  Total action: S_coord = {S_coord:.3f}")
print(f"\nComparison:")
print(f"  Theoretical (d=2): S ≈ {8.98 * 2**(-1.787):.3f}")
print(f"  π/8 quantum: S* ≈ {np.pi/8:.3f}")
print(f"\n✓ Quick start complete!")
print("="*50)